<a href="https://colab.research.google.com/github/xSakix/AI_colab_notebooks/blob/master/elasticsearch_install.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Elastic search in Collab

Had to install elastic search in colab for 'reasons' and this is the way it worked for me. Might be usefull for someone else also.

Works with 7.9.2. Probably could be run also with 7.14.0, but didn't have time to debug the issues. If you want, you can try and just run the instance under the 'elasticsearch' user to get the proper error log.

In [ ]:
#7.9.1 works with ES 7.9.2
!pip install -Iv elasticsearch==7.9.1

In [2]:
#download ES 7.92 and extract
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512 
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


In [3]:
# create user elasticsearch and group elasticsearch, under which will the ES instance be running
# ES can't run under root
!sudo useradd elasticsearch
!sudo grep elasticsearch /etc/passwd 
!sudo groupadd elasticsearch
!sudo usermod -a -G elasticsearch elasticsearch
!grep elasticsearch /etc/group

elasticsearch:x:1000:1000::/home/elasticsearch:/bin/sh
groupadd: group 'elasticsearch' already exists
elasticsearch:x:1000:elasticsearch


In [4]:
# change the directory rights to user:group 
!sudo chown elasticsearch:elasticsearch -R elasticsearch-7.9.2

In [5]:
#run ES instance as a daemon
%%bash --bg
sudo -H -u elasticsearch elasticsearch-7.9.2/bin/elasticsearch

Starting job # 0 in a separate thread.


In [6]:
# give time to start up
import time
time.sleep(20)

In [7]:
#print the process
%%bash

ps -ef | grep elastic

root         151     149  0 09:37 ?        00:00:00 sudo -H -u elasticsearch elasticsearch-7.9.2/bin/elasticsearch
elastic+     152     151 99 09:37 ?        00:00:20 /content/elasticsearch-7.9.2/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-13828270847204549773 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags

In [8]:
#test the instance
%%bash

curl -sX GET "localhost:9200/"

{
  "name" : "80196037624b",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "LQGhp_Y6TNGzHsw0lt4JEA",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [9]:
# test the python client/lib
from elasticsearch import Elasticsearch
es = Elasticsearch()
es.ping()

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


True